In [1]:

# This is lzl's snippet for diploma project
import math
import os,shutil
import random
import warnings
import gc

import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shapely
import seaborn as sns
from shapely.geometry import LineString, Point, Polygon
from shapely.wkt import dumps, loads
from tqdm import tqdm

import traProject as tp
from traProject.analysisFuns import statisticFuns as sfun
from traProject.basicFuns import CalFuns as cfun,VisFuns as vfun,DataProcessingFuns as dfun
from traProject.classificationFuns import model_feature_selection as mfs
import traProject.utils as tu

random.seed(1229)

# plt.style.use('seaborn')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

warnings.filterwarnings("ignore")

picOut='../../output/pic/paper/'


In [2]:
help(cfun.calV_link_new)

Help on function calV_link_new in module traProject.basicFuns.CalFuns:

calV_link_new(roadpath='../../output/paper/wuhan_road/WUHAN1_new.csv', trapath='../../output/paper/velocity/tra_all.csv', version=0, bufR=1e-05, minN=50, dropWifi=False, filename='tra_bylinkid_v', wfilename='数据质量问题路段', velocityName='velocity', outfile='', woutfile='', vlimit=[5, 120], toSingleFile=False)
    [按link分组 新]
    
    :param roadpath: [路网路径], defaults to '../../output/paper/wuhan_road/WUHAN1_new.csv'
    :type roadpath: str, optional
    :param trapath: [轨迹路径], defaults to '../../output/paper/velocity/tra_all.csv'
    :type trapath: str, optional
    :param version: [计算v方法], defaults to 0，{0:保留原始speed+wifi重新计算,1:全部重新计算,2:高acc保留+低acc和计算取均值}
    :type version: int, optional
    :param bufR: [缓冲区半径], defaults to 0.00001
    :type bufR: float, optional
    :param minN: [路段最小点数，数量过少路段不输出并记录至错误日志], defaults to 50
    :type minN: int, optional
    :param dropWifi: [是否剔除wifi], defaults to False
    :type dropWif

In [3]:
for vname in ['velocity','velocity1']:
    for v in [0,2]:
        cfun.calV_link_new(roadpath='../../output/paper/wuhan_road/WUHAN_new.csv', 
              trapath='../../output/paper/velocity/tra_all.csv',
              version=v, 
              bufR=2e-05, 
              minN=50, 
              dropWifi=False, 
              filename='tra_link_id_v',
              wfilename='problem_road_new',
              outfile=tu.pathCheck('../../output/paper/tra_linkid_new/'),
              velocityName=vname,
                vlimit=[0, 120],
                toSingleFile=True)

  0%|                                                                                   | 4/3684 [00:01<19:28,  3.15it/s]Exception ignored in: <function BaseGeometry.__del__ at 0x000001AC8B6F8D38>
Traceback (most recent call last):
  File "D:\progaramData\anaconda\envs\tra1\lib\site-packages\shapely\geometry\base.py", line 242, in __del__
    self.empty(val=None)
  File "D:\progaramData\anaconda\envs\tra1\lib\site-packages\shapely\geometry\base.py", line 227, in empty
    self._lgeos.GEOSGeom_destroy(self.__geom__)
KeyboardInterrupt: 
  0%|▍                                                                                 | 17/3684 [00:05<19:36,  3.12it/s]


KeyboardInterrupt: 

In [ ]:
versionList=[0,2]
PtsOrTraList=['Pts','Tra']
velocityNameList=['velocity','velocity1']
dropViaductList=[True,False]
dropWifiList=[True,False]

PtsOrTra='Pts'
dropViaduct=True
dropWifi=True
percentList=[0.5]
vTopNModeList=['1_0.5']
vTopNModeList.extend('2_%s'%i for i in percentList)

outfile='../../output/paper/FE_new/'

for version in versionList:
    sname='speed%s'%version
    for vTopNModestr in vTopNModeList:
        vTopNMode=int(vTopNModestr.split('_')[0])
        percent=float(vTopNModestr.split('_')[1])
        for velocityName in velocityNameList:
            versionName='%s_dropViaduct=%s_dropWifi=%s_vTopNmode=%s_velocity=%s'%(sname,dropViaduct,dropWifi,vTopNModestr,velocityName)
            fatherfiles=tu.getFileNames('../../output/paper/tra_linkid_new/tra_link_id_v%s_%s_minN=50_bufR=1e-05_dropWifi=False_velocity=%s/'%(version,PtsOrTra,velocityName))
            for fa in tqdm(fatherfiles):
                files=tu.getFileNames(fa+'/')
                for file in files:
                    tu.genDataSet1(file,
                                '../../output/paper/wuhan_road/WUHAN1_new.csv',
                                outfile=tu.pathCheck(outfile),
                                speedName=sname,
                                dname='tra_%s'%(versionName),
                                attrNames=['Sdis','Xdis','RC','viaduct'],
                                minPNum=50,
                                dropViaduct=dropViaduct,   
                                dropWifi=dropWifi,
                                vTopNMode=vTopNMode,
                                percent=percent
                                )

In [ ]:
mfs.main('../../output/paper/FE_new/','../../output/paper/models_new/')